Código del RS con CBR para el Trabajo de Grado de los estudiantes:

Edinson David León -
Luis Felipe Bastidas

In [42]:
from google.colab import files
uploaded = files.upload()

Saving Base_300.csv to Base_300 (5).csv


Importamos los casos que se encuentran en un archivo CSV.

In [63]:
import io
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
df_casos = pd.read_csv(io.BytesIO(uploaded['Base_300 (5).csv']))

La estructura de un caso se compone de dos partes, el **Problema** y la **Solución**.

En el CSV ya están definidos todos los datos de la base de casos, cuáles serán las *variables de entrada* o también denominados los datos del '**Problema**' y las *variables de salida* o datos de la '**Solución**'.

Estos datos se pueden representar de muchas formas: Datos numéricos reales, datos discretos, datos booleanos o datos simbólicos (palabras). Esto es importante definirlo al momento de construir el dataset o la base de casos. En este trabajo, se definen todos los datos de Problema, como valores numéricos reales; en cuanto a los datos de la Solución, tres son numéricos reales y los otros dos son valores discretos.

Para cada caso, se tienen 8 datos del Problema y 5 de Solución.

Los datos del Problema son: Densidad de siembra, Porcentaje de sombrío, Temporada, Humedad, Nitrógeno en el suelo, Fósforo en el suelo, Potasio en el suelo y pH en el suelo. Todos estos están en valores numéricos reales excepto el de Temporada, que tiene valores de *'Seca'* y *'Lluviosa'*, por lo que se procede a pasar estos valores a datos numéricos.

In [64]:
season = {'Seca': 0,'Lluviosa': 1, 'Normal': 0.5}
df_casos.Temporada = [season[item] for item in df_casos.Temporada]

Con esto, ya se tiene todo el DataFrame con datos numéricos.

In [65]:
print(df_casos.head())

   Id  Densidad  Sombrío  Temporada  Humedad  Nitrógeno  Fósforo  Potasio  \
0   1      4500     0.20        0.0      0.2         40      6.0       60   
1   2      4500     0.20        1.0      0.4         50      6.0       80   
2   3      4500     0.30        0.0      0.3         60      6.0      100   
3   4      4500     0.30        1.0      0.4         70      9.0      120   
4   5      4500     0.35        0.0      0.4         80      9.0      140   

    pH  Cond_N  Cond_P  Cond_K  Plus_A  Plus_B  
0  3.5       4       4       4       0       0  
1  4.0       3       3       3       1       0  
2  4.5       2       3       2       1       1  
3  5.0       2       3       2       1       1  
4  4.5       2       3       2       1       1  


Ahora, se procede a separar el DataFrame, y crear un nuevo DataFrame que sólo va a contener los datos de entrada.

In [66]:
df_problemas = df_casos.copy()
del df_problemas['Id'],
del df_problemas['Cond_N'],
del df_problemas['Cond_P'],
del df_problemas['Cond_K'],
del df_problemas['Plus_A'],
del df_problemas['Plus_B'],

print(df_problemas.head())

   Densidad  Sombrío  Temporada  Humedad  Nitrógeno  Fósforo  Potasio   pH
0      4500     0.20        0.0      0.2         40      6.0       60  3.5
1      4500     0.20        1.0      0.4         50      6.0       80  4.0
2      4500     0.30        0.0      0.3         60      6.0      100  4.5
3      4500     0.30        1.0      0.4         70      9.0      120  5.0
4      4500     0.35        0.0      0.4         80      9.0      140  4.5


Definimos un Array con datos de un nuevo caso, o mejor dicho, un **nuevo problema**.
Densidad 5700 plantas/ha
Sombrío 38%
Temporada lluviosa
Humedad suelo 75%
N suelo inorgánico 45 mg/Kg
N orgánico 0.45%
Materia orgánica 11.4%
P suelo 8 mg/kg
K suelo 125 mg/Kg (0.3197 cmol/Kg)
pH suelo 5

In [67]:
new_problem = np.array((5700, 0.38, 1, 0.75, 45, 8, 125, 5))
new_problem[0] = (new_problem[0] - 2500)/(7000-2500)
new_problem[1] = (new_problem[1] - 0)/(1-0)
new_problem[2] = (new_problem[2] - 0)/(1-0)
new_problem[3] = (new_problem[3] - 0)/(1-0)
new_problem[4] = (new_problem[4] - 0)/(225-0)
new_problem[5] = (new_problem[5] - 0)/(80-0)
new_problem[6] = (new_problem[6] - 0)/(546-0)
new_problem[7] = (new_problem[7] - 3)/(7-3)

Ante nuevos datos que entran en el sistema CBR, éstos deben ser procesados y analizados para encontrar los casos más similares al nuevo caso. Para este proceso se usa el algoritmo de Machine Learning llamado KNN. Este algortmo en CBR, consta de dos fases:



*   La primera fase consiste en la comparación del nuevo caso con los casos de la base de casos. Donde encontramos los k-casos más similares.
*   La segunda fase trata de la fusión, donde a partir de las soluciones de los k-casos más similares se determina una nueva solución para el nuevo caso.

El siguiente proceso es crear una nueva columna llamada *'Distancia'* que va a almacenar todas las distancias entre cada caso y el nuevo caso.


In [68]:
df_distance = df_problemas.assign(Distancia=0)
print(df_distance.head())

   Densidad  Sombrío  Temporada  Humedad  Nitrógeno  Fósforo  Potasio   pH  \
0      4500     0.20        0.0      0.2         40      6.0       60  3.5   
1      4500     0.20        1.0      0.4         50      6.0       80  4.0   
2      4500     0.30        0.0      0.3         60      6.0      100  4.5   
3      4500     0.30        1.0      0.4         70      9.0      120  5.0   
4      4500     0.35        0.0      0.4         80      9.0      140  4.5   

   Distancia  
0          0  
1          0  
2          0  
3          0  
4          0  


In [69]:
df_norm = pd.DataFrame(columns=['Densidad','Sombrío','Temporada','Humedad','Nitrógeno','Fósforo',
  'Potasio','pH','Distancia'])
for n in df_distance.index:
  df_norm.loc[n,'Densidad'] = (df_distance.loc[n,'Densidad'] - 2500)/(7000-2500)
  df_norm.loc[n,'Sombrío'] = (df_distance.loc[n,'Sombrío'] - 0)/(1-0)
  df_norm.loc[n,'Temporada'] = (df_distance.loc[n,'Temporada'] - 0)/(1-0)
  df_norm.loc[n,'Humedad'] = (df_distance.loc[n,'Humedad'] - 0)/(1-0)
  df_norm.loc[n,'Nitrógeno'] = (df_distance.loc[n,'Nitrógeno'] - 0)/(225-0)
  df_norm.loc[n,'Fósforo'] = (df_distance.loc[n,'Fósforo'] - 0)/(65-0)
  df_norm.loc[n,'Potasio'] = (df_distance.loc[n,'Potasio'] - 0)/(546-0)
  df_norm.loc[n,'pH'] = (df_distance.loc[n,'pH'] - 3)/(7-3)
  df_norm.loc[n,'Distancia'] = 0

df_norm.head()

,Densidad,Sombrío,Temporada,Humedad,Nitrógeno,Fósforo,Potasio,pH,Distancia
0,0.444444,0.2,0.0,0.2,0.177778,0.092308,0.10989,0.125,0
1,0.444444,0.2,1.0,0.4,0.222222,0.092308,0.14652,0.25,0
2,0.444444,0.3,0.0,0.3,0.266667,0.092308,0.18315,0.375,0
3,0.444444,0.3,1.0,0.4,0.311111,0.138462,0.21978,0.5,0
4,0.444444,0.35,0.0,0.4,0.355556,0.138462,0.25641,0.375,0


Se asume utilizar la Distancia Euclidiana para el cálculo de las distancias y se agregan en la columna creada anteriormente. Pero será una distancia euclidiana ponderada, ya que hay variables que van a tener un mayor peso que otras. A continucación explicamos los valores de peso que se consideran en el cálculo de las distancias:

* Peso = 1, variable nada relevante.
* Peso = 2, variable poco relevante.
* Peso = 3, variable algo relevante.
* Peso = 4, variable relevante.
* Peso = 5, variable muy relevante.

In [70]:
for n in df_norm.index:
  actual_case = df_norm.iloc[n].values
  total_sum = 0
  for m in range(0,8):
    if m == 0:
      factor = 2*(new_problem[m] - actual_case[m])*(new_problem[m] - actual_case[m])
      total_sum = total_sum + factor
    if m == 1:
      factor = 2*(new_problem[m] - actual_case[m])*(new_problem[m] - actual_case[m])
      total_sum = total_sum + factor
    if m == 2:
      factor = 5*(new_problem[m] - actual_case[m])*(new_problem[m] - actual_case[m])
      total_sum = total_sum + factor
    if m == 3:
      factor = 5*(new_problem[m] - actual_case[m])*(new_problem[m] - actual_case[m])
      total_sum = total_sum + factor
    if m == 4:
      factor = 2*(new_problem[m] - actual_case[m])*(new_problem[m] - actual_case[m])
      total_sum = total_sum + factor
    if m == 5:
      factor = 2*(new_problem[m] - actual_case[m])*(new_problem[m] - actual_case[m])
      total_sum = total_sum + factor
    if m == 6:
      factor = 2*(new_problem[m] - actual_case[m])*(new_problem[m] - actual_case[m])
      total_sum = total_sum + factor
    if m == 7:
      factor = 3*(new_problem[m] - actual_case[m])*(new_problem[m] - actual_case[m])
      total_sum = total_sum + factor
  e_distance = math.sqrt(total_sum)
  df_norm.loc[n,'Distancia'] = e_distance
print(df_norm.head())

   Densidad Sombrío Temporada Humedad Nitrógeno   Fósforo  Potasio     pH  \
0  0.444444     0.2       0.0     0.2  0.177778  0.092308  0.10989  0.125   
1  0.444444     0.2       1.0     0.4  0.222222  0.092308  0.14652   0.25   
2  0.444444     0.3       0.0     0.3  0.266667  0.092308  0.18315  0.375   
3  0.444444     0.3       1.0     0.4  0.311111  0.138462  0.21978    0.5   
4  0.444444    0.35       0.0     0.4  0.355556  0.138462  0.25641  0.375   

  Distancia  
0  2.677844  
1  1.010798  
2  2.495515  
3  0.891818  
4  2.419971  


El identificador de cada caso es importante, para posteriormente asociar los datos Problema con los datos de la Solución. Este identificador no se tiene en cuenta en el cálculo de las distancias pero ya calculadas esas distancias, se agrega nuevamente el identificador en cada caso.

In [71]:
df_ids = df_casos['Id']
#df_norm.insert(0,"Id",df_ids)
df_distance.insert(0,"Id",df_ids)
print(df_norm.head())

   Densidad Sombrío Temporada Humedad Nitrógeno   Fósforo  Potasio     pH  \
0  0.444444     0.2       0.0     0.2  0.177778  0.092308  0.10989  0.125   
1  0.444444     0.2       1.0     0.4  0.222222  0.092308  0.14652   0.25   
2  0.444444     0.3       0.0     0.3  0.266667  0.092308  0.18315  0.375   
3  0.444444     0.3       1.0     0.4  0.311111  0.138462  0.21978    0.5   
4  0.444444    0.35       0.0     0.4  0.355556  0.138462  0.25641  0.375   

  Distancia  
0  2.677844  
1  1.010798  
2  2.495515  
3  0.891818  
4  2.419971  


Así como se crea un DataFrame con todos los datos del Problema, también se crea un DataFrame con tods los datos de la Solución y el respectivo identificador en cada caso.

In [72]:
df_soluciones = df_casos.copy()
del df_soluciones['Densidad'],
del df_soluciones['Sombrío'],
del df_soluciones['Temporada'],
del df_soluciones['Humedad'],
del df_soluciones['Nitrógeno'],
del df_soluciones['Fósforo'],
del df_soluciones['Potasio']
del df_soluciones['pH']
print(df_soluciones.head())

   Id  Cond_N  Cond_P  Cond_K  Plus_A  Plus_B
0   1       4       4       4       0       0
1   2       3       3       3       1       0
2   3       2       3       2       1       1
3   4       2       3       2       1       1
4   5       2       3       2       1       1


 Para encontrar los k-casos más similares al nuevo caso, primero se determina el coeficiente k con mejores resultados, que según la evaluación realizada, es k=7

In [73]:
k = 7
print(k)

7


Se tiene dos DataFrames, uno contiene los datos del Problema y las distancias calculadas respecto al nuevo caso que entra al sistema. El segundo DataFrame contiene las soluciones de esos casos.

Se procede a juntar estos DataFrame en uno sólo otra vez.

In [74]:
new_df = pd.DataFrame(columns=['Id1','Densidad','Sombrío','Temporada','Humedad','Nitrógeno','Fósforo',
  'Potasio','pH','Distancia','Id2','Cond_N','Cond_P','Cond_K','Plus_A','Plus_B'])
join_array = np.array([])
for n in df_norm.index:
  join_array = np.append(df_distance.loc[n].values, df_soluciones.loc[n].values)
  #new_serie = pd.Series(data=join_array, name='x')
  #print('Array: ', join_array)
  temporal_df = pd.DataFrame([join_array],columns=['Id1','Densidad','Sombrío','Temporada','Humedad','Nitrógeno','Fósforo',
  'Potasio','pH','Distancia','Id2','Cond_N','Cond_P','Cond_K','Plus_A','Plus_B'])
  #print('Temporal_df: ', temporal_df)
  new_df = pd.concat([new_df, temporal_df])
  new_df.iloc[n,9] = df_norm.iloc[n,8]
new_df.head()

<ipython-input-74-e241f18abe7c>:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, temporal_df])


,Id1,Densidad,Sombrío,Temporada,Humedad,Nitrógeno,Fósforo,Potasio,pH,Distancia,Id2,Cond_N,Cond_P,Cond_K,Plus_A,Plus_B
0,1.0,4500.0,0.20,0.0,0.2,40.0,6.0,60.0,3.5,2.677844,1.0,4.0,4.0,4.0,0.0,0.0
0,2.0,4500.0,0.20,1.0,0.4,50.0,6.0,80.0,4.0,1.010798,2.0,3.0,3.0,3.0,1.0,0.0
0,3.0,4500.0,0.30,0.0,0.3,60.0,6.0,100.0,4.5,2.495515,3.0,2.0,3.0,2.0,1.0,1.0
0,4.0,4500.0,0.30,1.0,0.4,70.0,9.0,120.0,5.0,0.891818,4.0,2.0,3.0,2.0,1.0,1.0
0,5.0,4500.0,0.35,0.0,0.4,80.0,9.0,140.0,4.5,2.419971,5.0,2.0,3.0,2.0,1.0,1.0


Identificamos los k-casos más similares.

In [75]:
df_distance_sort = new_df.sort_values('Distancia')[:k]
df_distance_sort

,Id1,Densidad,Sombrío,Temporada,Humedad,Nitrógeno,Fósforo,Potasio,pH,Distancia,Id2,Cond_N,Cond_P,Cond_K,Plus_A,Plus_B
0,28.0,5500.0,0.40,1.0,0.60,60.0,6.0,180.0,5.0,0.382821,28.0,2.0,3.0,1.0,1.0,1.0
0,99.0,5200.0,0.32,1.0,0.65,65.0,9.0,65.0,5.6,0.438525,99.0,3.0,4.0,4.0,2.0,2.0
0,40.0,6000.0,0.20,1.0,0.70,80.0,18.0,140.0,5.0,0.445754,40.0,3.0,3.0,3.0,2.0,1.0
0,171.0,4900.0,0.50,1.0,0.65,75.0,10.0,165.0,5.2,0.448997,171.0,3.0,3.0,2.0,2.0,1.0
0,141.0,4600.0,0.36,1.0,0.75,85.0,3.0,65.0,4.8,0.470071,141.0,3.0,4.0,4.0,2.0,1.0
0,189.0,5300.0,0.10,1.0,0.75,35.0,5.0,175.0,4.6,0.473698,189.0,4.0,4.0,2.0,2.0,1.0
0,199.0,6500.0,0.18,1.0,0.70,30.0,8.0,105.0,5.6,0.485641,199.0,4.0,4.0,3.0,2.0,2.0


In [76]:
# Función que asigna los pesos para el promedio ponderado.
def get_peso(x):
  if x<0.5: return 10
  if x>=0.5 and x<1: return 8
  if x>=1 and x<1.5: return 6
  if x>=1.5 and x<2: return 4
  if x>=2 and x<2.5: return 2
  if x>=2.5 and x<3: return 1
  if x>=3: return 0

In [77]:
# La sección comentada es el método 2 que se utilizó para determinar una nueva solución (promedio ponderado),
# la cual se descartó posteriormente debido a que dió una precisión más baja.
"""
sum_N = 0
sum_pesos_N = 0
for m in range(0, len(df_distance_sort)):
  prod = get_peso(df_distance_sort.iloc[m,9])*df_distance_sort.iloc[m,11]
  sum_N = sum_N + prod
  sum_pesos_N = sum_pesos_N + get_peso(df_distance_sort.iloc[m,9])
Cond_N_prom = sum_N/sum_pesos_N
sum_P = 0
sum_pesos_P = 0
for m in range(0, len(df_distance_sort)):
  prod = get_peso(df_distance_sort.iloc[m,9])*df_distance_sort.iloc[m,12]
  sum_P = sum_P + prod
  sum_pesos_P = sum_pesos_P + get_peso(df_distance_sort.iloc[m,9])
Cond_P_prom = sum_P/sum_pesos_P
sum_K = 0
sum_pesos_K = 0
for m in range(0, len(df_distance_sort)):
  prod = get_peso(df_distance_sort.iloc[m,9])*df_distance_sort.iloc[m,13]
  sum_K = sum_K + prod
  sum_pesos_K = sum_pesos_K + get_peso(df_distance_sort.iloc[m,9])
Cond_K_prom = sum_K/sum_pesos_K
"""
Cond_N_prom = df_distance_sort['Cond_N'].mean()
Cond_P_prom = df_distance_sort['Cond_P'].mean()
Cond_K_prom = df_distance_sort['Cond_K'].mean()
Plus_A = df_distance_sort['Plus_A'].mode().values
Plus_B = df_distance_sort['Plus_B'].mode().values
#CBR_solution = np.array((Cond_N_prom[0], Cond_P_prom[0], Cond_K_prom[0], Plus_A[0], Plus_B[0]))
CBR_solution = np.array((Cond_N_prom, Cond_P_prom, Cond_K_prom, Plus_A[0], Plus_B[0]))
#round_CBR_solution = [round(num) for num in CBR_solution]
round_CBR_solution = CBR_solution


In [78]:
# Nuevo problema: (5500, 0.35, 1, 0.5, 40, 12, 196, 5.2)
#expert_solution = np.array((1, 0, 1, 1, 2))
CBR_solution = np.array((Cond_N_prom, Cond_P_prom, Cond_K_prom, Plus_A[0], Plus_B[0]))

In [79]:
#print('Solución experta del problema es', expert_solution)
print('Solución CBR del problema es', round_CBR_solution)

Solución CBR del problema es [3.14285714 3.57142857 2.71428571 2.         1.        ]


Con esta solución se procede a realizar la conversión de los datos, primero se normaliza el valor y luego se desnormaliza en la cantidad de fertilizante correspondiente a la densidad de siembra del cultivo.


In [80]:
cantidad_mínima_N_densidad_baja = 150
cantidad_mínima_N_densidad_normal = 180
cantidad_mínima_N_densidad_alta = 210
cantidad_mínima_P_densidad_baja = 30
cantidad_mínima_P_densidad_normal = 36
cantidad_mínima_P_densidad_alta = 42
cantidad_mínima_K_densidad_baja = 135
cantidad_mínima_K_densidad_normal = 165
cantidad_mínima_K_densidad_alta = 195
Densidad_nuevo_problema =  new_problem[0]*4500 + 2500

tasa_N_norm = (round_CBR_solution[0]-1)/3
tasa_P_norm = (round_CBR_solution[1]-1)/3
tasa_K_norm = (round_CBR_solution[2]-1)/3
if Densidad_nuevo_problema < 5000: Cantidad_fertilizante_N = tasa_N_norm*90 + cantidad_mínima_N_densidad_baja
if Densidad_nuevo_problema >= 5000 and Densidad_nuevo_problema <= 6000: Cantidad_fertilizante_N = tasa_N_norm*90 + cantidad_mínima_N_densidad_normal
if Densidad_nuevo_problema > 6000: Cantidad_fertilizante_N = tasa_N_norm*90 + cantidad_mínima_N_densidad_alta
if Densidad_nuevo_problema < 5000: Cantidad_fertilizante_P = tasa_P_norm*18 + cantidad_mínima_P_densidad_baja
if Densidad_nuevo_problema >= 5000 and Densidad_nuevo_problema <= 6000: Cantidad_fertilizante_P = tasa_P_norm*18 + cantidad_mínima_P_densidad_normal
if Densidad_nuevo_problema > 6000: Cantidad_fertilizante_P = tasa_P_norm*18 + cantidad_mínima_P_densidad_alta
if Densidad_nuevo_problema < 5000: Cantidad_fertilizante_K = tasa_K_norm*90 + cantidad_mínima_K_densidad_baja
if Densidad_nuevo_problema >= 5000 and Densidad_nuevo_problema <= 6000: Cantidad_fertilizante_K = tasa_K_norm*90 + cantidad_mínima_K_densidad_normal
if Densidad_nuevo_problema > 6000: Cantidad_fertilizante_K = tasa_K_norm*90 + cantidad_mínima_K_densidad_alta

In [81]:
print('Las recomendaciones finales son (en Kg/ha por año): ', Cantidad_fertilizante_N, 'de fertilizante de N')
print(Cantidad_fertilizante_P, ' de fertilizante de P')
print(Cantidad_fertilizante_K, ' de fertilizante de K')

Las recomendaciones finales son (en Kg/ha por año):  244.28571428571428 de fertilizante de N
51.42857142857143  de fertilizante de P
216.42857142857144  de fertilizante de K
